# Sample RAG Agent Walkthrough

This notebook will walk users through setting up a sample RAG Agent and run it against the Hugging Face 'rag-mini-wikipedia' dataset (https://huggingface.co/datasets/rag-datasets/rag-mini-wikipedia)

#### Ensure the latest version of boto3 is shown below

##### If not then run through setup_environment.ipynb in the 0-Notebook-environment/ folder

In [49]:
!pip freeze | grep boto3

boto3==1.36.24


#### Load in environment variables to notebook

In [50]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [51]:
%run $IMPORTS_PATH

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/19/25 21:05:09] INFO     Found credentials from IAM Role:                                   ]8;id=448325;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=889276;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Successfully imported necessary libraries into notebook


# Prerequisites

This notebook assumes that you have gone through the notebook environment setup in the 0-Notebook-environment/ folder and deployed Langfuse v3 to your local AWS environment

# Data Preparation

### Retrieve Hugging Face 'rag-mini-wikipedia'

In [52]:
# Retrieves dataset corpus using Datasets Python library

ds_corpus = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

### Write text corpus to file and upload to Amazon S3 to use as data source for knowledge base

In [55]:
# Write whole corpus to a .txt file

with open('1-Sample-rag-agent/mini_wiki.txt', 'w') as f:
    f.write(str(ds_corpus['passages']['passage']))

Note: You can now view the whole Wikipedia corpus in mini_wiki.txt

In [57]:
# Create Amazon S3 bucket and upload .txt. file to Amazon S3 bucket

s3_client = boto3.client('s3')

bucket_name = f"rag-mini-wikipedia-data-{account_id}"

s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={
        'LocationConstraint': region
    }
)

{'ResponseMetadata': {'RequestId': 'F4BSJFDDV12AQYDT',
  'HostId': 'EjQNdiJPAx1wNfP8jhI7bHhpQ1d4Z2K5qUxQbLnyyHumNg7//9eX+B/eHtBJd964ve+DL8qWQio=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'EjQNdiJPAx1wNfP8jhI7bHhpQ1d4Z2K5qUxQbLnyyHumNg7//9eX+B/eHtBJd964ve+DL8qWQio=',
   'x-amz-request-id': 'F4BSJFDDV12AQYDT',
   'date': 'Wed, 19 Feb 2025 21:19:55 GMT',
   'location': 'http://rag-mini-wikipedia-data-761018876800.s3.amazonaws.com/',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': 'http://rag-mini-wikipedia-data-761018876800.s3.amazonaws.com/'}

In [59]:
# Place .txt corpus in S3 bucket

s3_client.upload_file('1-Sample-rag-agent/mini_wiki.txt', bucket_name, 'mini_wiki.txt')

### Create Bedrock Knowledge Base
#### Follow the steps below to create a Bedrock Knowledge Base in the AWS Console manually

Step 1: Navigate to the 'Amazon Bedrock' service in the AWS Console and navigate to the 'Knowledge Bases' section

Step 2: Click 'Create' and select 'Knowledge Base with vector store'

Step 3: Name the Knowledge Base 'mini-wiki-kb' and select the Amazon S3 data source radio button

Step 4: Name the data source 'mini-wiki-data' and select the S3 bucket file 'mini_wiki.txt' that was uploaded, 
        e.x. s3://rag-mini-wikipedia-data-XXXXXXXXXXXX/mini_wiki.txt

Step 5: Select the 'Titan Embeddings G1 - Text V1.2' embedding model and create Amazon OpenSearch Serverless with the quick create option

Step 6: Now create the knowledge base

Step 7: Manually sync the data source with the knowledge base by clicking on the data source and selecting 'Sync'

In [76]:
# Fetch knowledge base ARN

bedrock_agent_client = boto3.client("bedrock-agent", region)

# Call the list_knowledge_bases method
response = bedrock_agent_client.list_knowledge_bases()
kb_id = None

# Iterate through knowledge bases and find needed one
if 'knowledgeBaseSummaries' in response:
    for kb in response['knowledgeBaseSummaries']:
        if 'rag-mini-wiki-kb-official' in kb['name']:
            kb_id = kb['knowledgeBaseId']
            break

response = bedrock_agent_client.get_knowledge_base(
    knowledgeBaseId=kb_id
)

kb_arn = [response['knowledgeBase']['knowledgeBaseArn']]

print(f"Found knowledge base ARN: {})

Found knowledge base ID: LLOLPSDWLU


### Create RAG Agent

In [79]:
agent_foundation_model = ["anthropic.claude-3-5-sonnet-20241022-v2:0"]
agent_name = 'sample-rag-agent-test'
agent_description = "RAG agent to run against the Hugging Face 'rag-mini-wikipedia' dataset"
agent_instruction = """Use the associated knowledge base to answer questions."""

In [80]:
agents = AgentsForAmazonBedrock()

# Add parameter with kb_arn based on newly created knowledge base
rag_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arn,
    code_interpretation=False,
    verbose=False
)

rag_agent

('4YAR7X7KGG',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:761018876800:agent-alias/4YAR7X7KGG/TSTALIASID')

### Test RAG Agent

#### Invoke Sample RAG Agent Test Alias to see that it answers question properly

In [84]:
# Ask example question to agent

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

test_query = "Who suggested Lincoln grow a beard?"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=rag_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

Request sent to Agent:
{'ResponseMetadata': {'RequestId': 'ec31f3e7-8457-40c2-9517-c516b686c922', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 20 Feb 2025 00:39:17 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'ec31f3e7-8457-40c2-9517-c516b686c922', 'x-amz-bedrock-agent-session-id': '1dc66c1a-ef23-11ef-b3d3-063ee2632e5d', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '1dc66c1a-ef23-11ef-b3d3-063ee2632e5d', 'completion': <botocore.eventstream.EventStream object at 0x7f9d95b9f640>}
Agent processing query now
Agent Answer: Grace Bedell, an 11-year-old girl from Westfield, New York, wrote a letter to Abraham Lincoln in 1860 suggesting he grow a beard to improve his appearance and help him win the presidency. Lincoln responded to her letter and began growing his now-famous beard.


#### Prepare agent and create alias for use with evaluation framework

In [81]:
rag_agent_id = rag_agent[0]
rag_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{rag_agent_id}"

rag_agent_id, rag_agent_arn

('4YAR7X7KGG', 'arn:aws:bedrock:us-west-2:761018876800:agent/4YAR7X7KGG')

### Create input .json file for all RAG using ground truth provided by Hugging Face dataset

In [66]:
# Create input json data file for evaluation framework and place so it can be run by evaluation framework by user

ds_qa = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")

# Create the dictionary structure
input_data_dict = {
    "RAG": []
}

# Iterate through all elements in the test split and add them to the RAG list
for data in ds_qa['test']:
    
    # Incorporate evaluation_type tag in qa_pair and also use for biomarker solution trajectories
    qa_pair = {
        "question_id": data['id'],
        "question": data['question'],
        "ground_truth": data['answer'],
        "evalution_type": "RAG"
    }   
    input_data_dict["RAG"].append(qa_pair)

# Save to JSON file
with open('1-Sample-rag-agent/rag_data_file_auto.json', 'w', encoding='utf-8') as f:
    json.dump(input_data_dict, f, indent=4, ensure_ascii=False)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


In [67]:
# TODO: Write code to place input file so that evaluation framework can use
# Lead user to now run evaluation framework against .json file